**SA463A &#x25aa; Data Wrangling and Visualization &#x25aa; Fall 2020 &#x25aa; Foraker and Uhan**

# Lesson 16. Working with Missing Values in Pandas

## In this lesson...

- We learned a little bit about missing values back in Lesson 12


- In this lesson, we'll learn about working with missing values in Pandas in more detail &mdash; quirks and all

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Setup

* Let's start by importing Pandas and NumPy:

In [1]:
import pandas as pd
import numpy as np

* For this lesson, we'll work with a small example dataset, in the CSV file `data/toy.csv` in the same folder as this notebook

In [2]:
toy_df = pd.read_csv(
    'data/toy.csv',
    parse_dates=['datetimes']
)

- Note that `parse_dates=...` takes a list of columns to be parsed as a datetime data type (dtype)
    - See [the documentation for `pd.read_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) for details


* Let's see what this dataset looks like:

In [3]:
toy_df.head()

,floats,strings,integers,datetimes
0,4.3,abc,3.0,NaT
1,9.1,def,9.0,2020-02-01
2,NaN,def,NaN,2020-03-01
3,NaN,abc,7.0,2020-04-01
4,7.9,NaN,-99.0,NaT


- Looking at the raw CSV file, we see that the `integers` column contains integers or missing values


- However, note that the `integers` column, consisting of integers, was read in as a float


- Unfortunately, Pandas cannot (easily) handle a Series of integers with NaN values
    - Actually, there is a way to do this, but it is currently experimental

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Values considered "missing"

- We saw in an earlier lesson that Pandas uses `NaN` to mark values that are "missing" or "not available" or "NA"


- Pandas has other markers for missing values, such as `NaT` for datetime dtypes

    
- Good news: `.isna()` and `.notna()` will detect NA values, no matter the marker used

In [4]:
# Solution
toy_df.query('datetimes.isna()', engine='python')

,floats,strings,integers,datetimes
0,4.3,abc,3.0,NaT
4,7.9,NaN,-99.0,NaT


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Sorting with missing data

- Recall that when we use `.sort_values()` to sort a DataFrame by the values of one of its columns, the missing values get placed at the end


- For example, let's see what happens when we sort our DataFrame by `floats`:

In [5]:
# Solution
toy_df.sort_values('floats')

,floats,strings,integers,datetimes
0,4.3,abc,3.0,NaT
4,7.9,NaN,-99.0,NaT
1,9.1,def,9.0,2020-02-01
2,NaN,def,NaN,2020-03-01
3,NaN,abc,7.0,2020-04-01


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Computations with missing values

- Arithmetic operations with missing values results in NA


- For example, let's see what happens when we add the `floats` and `integers` columns together:

In [6]:
# Solution
toy_df['floats'] + toy_df['integers']

0     7.3
1    18.1
2     NaN
3     NaN
4   -91.1
dtype: float64

- Happily, many of the built-in Pandas methods that perform descriptive statistics and computational methods, like the *reduction*/*aggregation* and _"same size"_ methods from Lessons 14 and 15, are written to account for missing values


- For example, when using `.sum()`, NA values are skipped by default


- To illustrate, let's see what happens when we sum the values in the `floats` and `integer` columns across the rows:

In [7]:
# Solution
toy_df[['floats', 'integers']].sum()

floats      21.3
integers   -80.0
dtype: float64

- Note that [the documentation for the `sum` DataFrame method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sum.html) describes the `skipna=...` keyword argument, which is `True` by default


- This is also the case for Series/DataFrame methods like `max`, `mean`, `median`, `min`, `mode`, `std`, `var`, etc.

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Groupby with missing values

- NA values in groupby operations are automatically excluded


- For example, let's group our dataset by the values of `strings` and compute the mean of `floats` for each group:

In [8]:
# Solution
(
    toy_df
    .groupby('strings')
    .agg(
        mean_floats=('floats', 'mean')
    )
    .reset_index()
)

,strings,mean_floats
0,abc,4.3
1,def,9.1


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Filling missing values

- Sometimes ignoring or skipping NA values is the way to go, as `sum` and other Pandas built-in methods do by default


- Other times, you may want to fill NA values with something else that makes sense


- We can use the `.fillna()` Series/DataFrame method to accomplish this


- For example, we can replace the NA values in the `floats` column with 0:

In [9]:
# Solution
toy_df['floats'].fillna(0)

0    4.3
1    9.1
2    0.0
3    0.0
4    7.9
Name: floats, dtype: float64

- Note that `.fillna()`, when applied to a Series, returns a Series
    - When applied to a DataFrame, `.fillna()` returns a DataFrame


- Instead of using 0, we can replace the NA values in the `floats` column with the mean of the non-NA values:

In [10]:
# Solution
toy_df['floats'].fillna(toy_df['floats'].mean())

0    4.3
1    9.1
2    7.1
3    7.1
4    7.9
Name: floats, dtype: float64

- We can also use `fillna()` to forward-fill or back-fill values, like this:

In [11]:
toy_df['floats'].fillna(method='ffill')

0    4.3
1    9.1
2    9.1
3    9.1
4    7.9
Name: floats, dtype: float64

In [12]:
toy_df['floats'].fillna(method='bfill')

0    4.3
1    9.1
2    7.9
3    7.9
4    7.9
Name: floats, dtype: float64

- See [the documentation for `.fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) for details

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Dropping rows or columns with missing values

- If you just want to exclude rows or columns with missing values, you can use `.dropna()`


- By default, `.dropna()` returns a new DataFrame with all the rows containing NA values dropped:

In [13]:
toy_df.dropna()
# toy_df.dropna(axis='rows') does the same thing

,floats,strings,integers,datetimes
1,9.1,def,9.0,2020-02-01


- We can obtain a new DataFrame with all the columns containing NA values dropped by using the `axis='columns'` keyword argument:

In [14]:
# Solution
toy_df.dropna(axis='columns')

""
0
1
2
3
4


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Replacing values with NA

- Sometimes, missing values are encoded with a **sentinel value**: a special value designated for missing values


- For example, suppose `-99` is a sentinel value in our dataset: we see that the `integers` column contains the value `-99.0`, which was originally `-99` in the CSV file:

In [15]:
toy_df

,floats,strings,integers,datetimes
0,4.3,abc,3.0,NaT
1,9.1,def,9.0,2020-02-01
2,NaN,def,NaN,2020-03-01
3,NaN,abc,7.0,2020-04-01
4,7.9,NaN,-99.0,NaT


- We can replace values of `-99.0` with a proper NA value marker using the `replace()` Series/DataFrame method, like this:

In [16]:
# Solution
toy_df['integers'].replace(-99.0, np.nan)

0    3.0
1    9.0
2    NaN
3    7.0
4    NaN
Name: integers, dtype: float64

- Note that `.replace()` returns a new Series when applied to a Series
    - `.replace()` returns a new DataFrame when applied to a DataFrame
    
    
- So, we can add a new column to our DataFrame with the revised `integers` column, like this:

In [17]:
# Solution
toy_df.assign(
    integers_nan=lambda x: x['integers'].replace(-99.0, np.nan)
)

,floats,strings,integers,datetimes,integers_nan
0,4.3,abc,3.0,NaT,3.0
1,9.1,def,9.0,2020-02-01,9.0
2,NaN,def,NaN,2020-03-01,NaN
3,NaN,abc,7.0,2020-04-01,7.0
4,7.9,NaN,-99.0,NaT,NaN


- Alternately, we can replace the `NaN` values in the `integers` column with a sentinel value


- Then, we can convert the `integers` column to be a proper column of `int` dtypes with the `.astype()` method


- Like this:

In [18]:
# Solution
toy_df['integers'].replace(np.nan, -99).astype(int)

0     3
1     9
2   -99
3     7
4   -99
Name: integers, dtype: int64

- Here is [the documentation for the `.replace()` Series method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html)


- Here is [the documentation for the `.astype()` Series method](https://pandas.pydata.org/docs/reference/api/pandas.Series.astype.html)

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Problems

**Problem 0.**
In the same folder as this notebook, there is a zipped CSV file `data/nycflights13_flights.csv.zip`, containing the same nycflights13 dataset we used in previous lessons. Read the CSV file into a DataFrame. Display the top 5 rows of the DataFrame.

In [19]:
# Solution
df = pd.read_csv('data/nycflights13_flights.csv.zip')
df.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


**Problem 1.** Are there any rows with missing `time_hour` values?

In [20]:
# Solution
df.query('time_hour.isna()', engine='python').shape

(0, 19)

**Problem 2.**
Drop all rows with missing values in the nycflights13 dataset. How many rows remain?

In [21]:
# Solution
df.dropna().shape

(327346, 19)

**Problem 3.** Compute the average arrival delay for each month, when you 

1. assume that any missing arrival delay has a value of 0, and 
2. omit all missing arrival delays. 

Compare the values you get from both computations. Do they make sense?

In [22]:
# Solution
(
    df
    .assign(
        arr_delay_0=lambda x: x['arr_delay'].fillna(0)
    )
    .groupby(['year', 'month'])
    .agg(
        mean_arr_delay=('arr_delay', 'mean'),
        mean_arr_delay_0=('arr_delay_0', 'mean')
    )
    .reset_index()
)

,year,month,mean_arr_delay,mean_arr_delay_0
0,2013,1,6.129972,5.992409
1,2013,2,5.613019,5.311571
2,2013,3,5.807577,5.619859
3,2013,4,11.176063,10.873879
4,2013,5,3.521509,3.439818
5,2013,6,16.481330,15.799738
6,2013,7,16.711307,16.068411
7,2013,8,6.040652,5.923040
8,2013,9,-4.018364,-3.936172
9,2013,10,-0.167063,-0.165496


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Notes and sources

- From the [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html):
    - [Working with missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)